## DeepEval

Custom evaluator class for Llama3: https://docs.confident-ai.com/docs/guides-using-custom-llms

In [3]:
import transformers
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

from deepeval.models import DeepEvalBaseLLM

class CustomLlama3_8B(DeepEvalBaseLLM):
    def __init__(self): 

        model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct",
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct"
        )

        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_length=2500,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        return pipeline(prompt)

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Llama-3.1 8B"

In [4]:
custom_llm = CustomLlama3_8B()
print(custom_llm.generate("Write me a joke"))

Trying to resume download...


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs-us-1.hf.co\', port=443): Max retries exceeded with url: /repos/75/15/75158e9667a787ec0ccbd8d18412c5d9a5f7326c543b803c4932dbc95951ed30/09d433f650646834a83c580877bd60c6d1f88f7755305c12576b5c7058f9af15?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1732365010&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjM2NTAxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzc1LzE1Lzc1MTU4ZTk2NjdhNzg3ZWMwY2NiZDhkMTg0MTJjNWQ5YTVmNzMyNmM1NDNiODAzYzQ5MzJkYmM5NTk1MWVkMzAvMDlkNDMzZjY1MDY0NjgzNGE4M2M1ODA4NzdiZDYwYzZkMWY4OGY3NzU1MzA1YzEyNTc2YjVjNzA1OGY5YWYxNT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=TR7iW5MLJEshAhoelNaiBTPr5KjfJQDBV-Ni-5U9dk8fUo4O8dQjmyKeDHCKQyTDPCVxdI8ES~WO0CDzwkhu6VJEjdeNhF~AuDJrBTDRijfTLYw5NY-44zUXXJMZq7uauSWWQZ4YHtXbWuEtU8j-N0WVqxcXtfcjT